In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding
from keras.preprocessing.sequence import pad_sequences
df=pd.read_csv(r"B:\project\newdataset\chunk_5.csv")
df = df.drop(columns=['Unnamed: 0'])
df

,text,generated
0,"Dear TEACHER_game,\n\nHave you ever been comin...",0.0
1,In this Essay I'm going to bl Explaining about...,0.0
2,Do you think there's a real alien artifact on ...,0.0
3,One way we know that it been now created of al...,0.0
4,What is the electoral college? Is when the peo...,0.0
...,...,...
17983,The face of Mars has been Mystery for many yea...,0.0
17984,Students should non have no participate in ext...,0.0
17985,What if you went to school without actually st...,0.0
17986,Although the primary purpose of summer break i...,0.0


In [2]:
from transformers import LongformerTokenizer
tokenizer=LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
essays=df['text'].tolist()
tokens=tokenizer(essays,return_tensors='pt',padding=True,truncation=True)

In [3]:
x=tokens['input_ids']
y=df['generated']
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=42)
input_ids_size = tokens['input_ids'].size()
attention_mask_size = tokens['attention_mask'].size()
print("Input IDs size:", input_ids_size)
print("Attention Mask size:", attention_mask_size)
# Assuming 'tokens' is the result of tokenizer
unique_tokens = set(token.item() for seq in tokens['input_ids'] for token in seq)
vocabulary_size = len(unique_tokens)

print("Vocabulary Size:", vocabulary_size)
# Assuming 'input_ids' is the first element in xtrain
max_sequence_length_x = max(len(seq) for seq in xtrain)

# Print or use the max_sequence_length_x in your pad_sequences and Embedding layer
print("Max Sequence Length:", max_sequence_length_x)


Input IDs size: torch.Size([17988, 1683])
Attention Mask size: torch.Size([17988, 1683])
Vocabulary Size: 29800
Max Sequence Length: 1683


In [4]:
model=Sequential()
input_length=1683
xtrain_np=xtrain.numpy()
model.add(Embedding(input_dim=52000,output_dim=128,input_length=input_length))
model.add(LSTM(64))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(xtrain_np,ytrain,epochs=5,batch_size=32)



Epoch 1/5


450/450 [==============================] - 235s 519ms/step - loss: 0.0143 - accuracy: 0.9978
Epoch 2/5
450/450 [==============================] - 229s 509ms/step - loss: 3.9939e-05 - accuracy: 1.0000
Epoch 3/5
450/450 [==============================] - 230s 511ms/step - loss: 1.9065e-05 - accuracy: 1.0000
Epoch 4/5
450/450 [==============================] - 230s 510ms/step - loss: 1.2177e-05 - accuracy: 1.0000
Epoch 5/5
450/450 [==============================] - 240s 533ms/step - loss: 8.3953e-06 - accuracy: 1.0000


In [5]:
xtest_np=xtest.numpy()
model.evaluate(xtest_np,ytest)

113/113 [==============================] - 17s 145ms/step - loss: 7.0513e-06 - accuracy: 1.0000


[7.051279681036249e-06, 1.0]

In [6]:
model.save("newmodel5.keras")